In [57]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge #ordinary linear regression + w/ ridge regularization

In [58]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Oct  4 16:25:31 2018

@author: mahlborg

File: Formatted_To_Consolidated
"""

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 550)
pd.set_option('display.max_columns', 20)

#from LBC_Viz import *


Country_Metrics_Annual = pd.read_csv('/Users/mahlborg/Desktop/Metis/LBC/Country_Metrics_Annual.csv')


countries_mapped_to_currencies = pd.read_csv('/Users/mahlborg/Desktop/Metis/LBC/Supporting Data/countries_mapped_to_currencies.csv')
countries_mapped_to_currencies.columns = ['COUNTRY', 'CURRENCY', 'TICKER']
countries_mapped_to_currencies.loc[len(countries_mapped_to_currencies)] = ["Central Africa", "Central Africa Franc", "XAF"]
set1 = set(countries_mapped_to_currencies['TICKER'])
#GDP_PPP_PC column creation
GDP_PPP_PC = pd.read_csv('/Users/mahlborg/Desktop/Metis/LBC/Supporting Data/GDP_PPP.csv')
GDP_PPP_PC.columns = ['COUNTRY', 'GDP_PPP_PC']
GDP_PPP_PC['GDP_PPP_PC'] = GDP_PPP_PC['GDP_PPP_PC'].str.replace(',', '')
GDP_PPP_PC['GDP_PPP_PC'] = pd.to_numeric(GDP_PPP_PC['GDP_PPP_PC'], errors = 'coerce')
GDP_PPP_PC.loc[len(GDP_PPP_PC)] = ["West Africa", 2077]
GDP_PPP_PC.loc[len(GDP_PPP_PC)] = ["Central Africa", 4107]
GDP_PPP_PC.loc[len(GDP_PPP_PC)] = ["European Union", 39970]
#print(GDP_PPP_PC)

combined_df = pd.merge(countries_mapped_to_currencies, GDP_PPP_PC , on='COUNTRY')
set2 = set(combined_df['TICKER'])
#COUNTRY_POP column creation
#print(combined_df)
COUNTRY_POP = pd.read_csv('/Users/mahlborg/Desktop/Metis/LBC/Supporting Data/COUNTRY_POP.csv')
COUNTRY_POP.columns = ['COUNTRY', 'POPULATION']
COUNTRY_POP['POPULATION'] = COUNTRY_POP['POPULATION'].str.replace(',', '')
COUNTRY_POP['POPULATION'] = pd.to_numeric(COUNTRY_POP['POPULATION'], errors = 'coerce')
COUNTRY_POP.loc[len(COUNTRY_POP)] = ["European Union", 509000000]
COUNTRY_POP.loc[len(COUNTRY_POP)] = ["West Africa", 129000000]
COUNTRY_POP.loc[len(COUNTRY_POP)] = ["Central Africa", 52000000]
#print(COUNTRY_POP)
combined_df = pd.merge(combined_df, COUNTRY_POP , on='COUNTRY')
set3 = set(combined_df['TICKER'])
#Corruption Column creation
CORRUPTION = pd.read_csv('/Users/mahlborg/Desktop/Metis/LBC/Supporting Data/2018_Corruption_Scores.csv')
CORRUPTION.columns = ['COUNTRY', 'CORRUPTION_SCORE']
#print(CORRUPTION)
CORRUPTION['CORRUPTION_SCORE'] = pd.to_numeric(CORRUPTION['CORRUPTION_SCORE'], errors = 'coerce')
CORRUPTION.loc[len(CORRUPTION)] = ["West Africa", 33]
CORRUPTION.loc[len(CORRUPTION)] = ["Central Africa", 23]
CORRUPTION.loc[len(CORRUPTION)] = ["European Union", 70]


combined_df = pd.merge(combined_df, CORRUPTION , on='COUNTRY')
set4 = set(combined_df['TICKER'])
#print(combined_df)
#Inflation Column creation
INFLATION = pd.read_csv('/Users/mahlborg/Desktop/Metis/LBC/Supporting Data/2018_Inflation.csv')
INFLATION.columns = ['COUNTRY', 'INFLATION']
INFLATION['INFLATION'] = pd.to_numeric(INFLATION['INFLATION'], errors = 'coerce')
INFLATION.loc[len(INFLATION)] = ["West Africa", 2.2]
INFLATION.loc[len(INFLATION)] = ["Central Africa", 8.9]
combined_df = pd.merge(combined_df, INFLATION , on='COUNTRY')
set5 = set(combined_df['TICKER'])
#Internet_PEN column creation
INTERNET_PEN = pd.read_csv('/Users/mahlborg/Desktop/Metis/LBC/Supporting Data/2017_Internet_Penetration.csv')
INTERNET_PEN.columns = ['COUNTRY', 'INTERNET_PEN']
INTERNET_PEN['INTERNET_PEN'] = pd.to_numeric(INTERNET_PEN['INTERNET_PEN'], errors = 'coerce')
INTERNET_PEN.loc[len(INTERNET_PEN)] = ["West Africa", 13.4]
INTERNET_PEN.loc[len(INTERNET_PEN)] = ["Central Africa", 8.5]
INTERNET_PEN.loc[len(INTERNET_PEN)] = ["European Union", 79.8]
combined_df = pd.merge(combined_df, INTERNET_PEN , on='COUNTRY')
set6 = set(combined_df['TICKER'])
#Average Transaction Size column creation
Country_Metrics_Annual['AVG_TX_SIZE_USD'] = Country_Metrics_Annual['USD_EQUIV'] / (Country_Metrics_Annual['TX'][Country_Metrics_Annual['TX'] > 0])
#print(Country_Metrics_Annual)

Country_Metrics_Annual = pd.merge(Country_Metrics_Annual, combined_df , on='TICKER')

#UPEP column creation
Country_Metrics_Annual['UPEP'] = ((Country_Metrics_Annual['USD_EQUIV'] / Country_Metrics_Annual['POPULATION']) / Country_Metrics_Annual['GDP_PPP_PC']) * 1000
#Country_Metrics_Annual['UPEP'] = Country_Metrics_Annual.groupby(pd.Grouper(freq='A')).UPEP.mean()

In [59]:
Country_Metrics_Annual.head()

,DATE,TX,VOLUME,USD_EQUIV,TICKER,AVG_TX_SIZE_USD,COUNTRY,CURRENCY,GDP_PPP_PC,POPULATION,CORRUPTION_SCORE,INFLATION,INTERNET_PEN,UPEP
0,2015-12-31,6,0.856600,283.217475,DZD,47.202912,Algeria,Algerian dinar,15150,41318142,33,5.5,42.95,4.524459e-07
1,2016-12-31,6,0.396600,237.650071,DZD,39.608345,Algeria,Algerian dinar,15150,41318142,33,5.5,42.95,3.796510e-07
2,2017-12-31,8,0.285186,354.632884,DZD,44.329110,Algeria,Algerian dinar,15150,41318142,33,5.5,42.95,5.665335e-07
3,2014-12-31,6,2.575395,1453.079724,NAD,242.179954,Namibia,Namibian dollar,11528,2533794,51,6.0,31.03,4.974669e-05
4,2015-12-31,8,1.786600,427.565677,NAD,53.445710,Namibia,Namibian dollar,11528,2533794,51,6.0,31.03,1.463786e-05


In [60]:
Country_Metrics_Annual = Country_Metrics_Annual[Country_Metrics_Annual['DATE'] == '2017-12-31']

In [61]:
Country_Metrics_Annual.head()

,DATE,TX,VOLUME,USD_EQUIV,TICKER,AVG_TX_SIZE_USD,COUNTRY,CURRENCY,GDP_PPP_PC,POPULATION,CORRUPTION_SCORE,INFLATION,INTERNET_PEN,UPEP
2,2017-12-31,8,0.285186,3.546329e+02,DZD,44.329110,Algeria,Algerian dinar,15150,41318142,33,5.5,42.95,5.665335e-07
10,2017-12-31,4145,57.416249,3.914083e+05,GHS,94.429020,Ghana,Ghanaian cedi,4605,28833629,40,11.8,34.67,2.947821e-03
16,2017-12-31,1212,281.119581,7.410789e+05,EGP,611.451217,Egypt,Egyptian pound,12994,97553151,32,13.5,41.25,5.846289e-04
21,2017-12-31,320,34.754207,1.282491e+05,BGN,400.778506,Bulgaria,Bulgarian lev,21578,7084571,43,1.1,59.83,8.389376e-04
23,2017-12-31,6024,1488.050230,9.736125e+06,PAB,1616.222570,Panama,United States dollar,24262,4098587,37,1.6,54.00,9.790962e-02


In [62]:
Country_Metrics_Annual = Country_Metrics_Annual.drop(['DATE', 'TICKER', 'COUNTRY', 'CURRENCY'], axis=1)

In [63]:
X, y = Country_Metrics_Annual.drop('UPEP',axis=1), Country_Metrics_Annual['UPEP']

In [64]:
X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=10) #hold out 20% of the data for final testing
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.25, random_state=3) #hold out 20% for validation

In [65]:
lm = LinearRegression()

In [66]:
#Feature scaling for train, val, and test so that we can run our ridge model on each
scaler = StandardScaler()

In [67]:
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [68]:
lm.fit(X_train, y_train)
print('Linear Regression val R^2: %.3f', lm.score(X_val, y_val))

Linear Regression val R^2: %.3f 0.7334674446720959


In [69]:
diagnostic_plot(x, y)


NameError: name 'diagnostic_plot' is not defined